In [ ]:
# use python eval sometimes. great trickdefining a class and operator overloading 

In [16]:
import aoc
f = open('input.txt')
lines = [line.rstrip('\n') for line in f]
lines[0]

'4 * 3 * (2 * 6 + (3 * 3 + 8 + 7 + 2) + 9 + 8) * 4 * 6'

In [53]:
# part 1
def evaluate(line):
    ans = 0
    firstop = None
    operator = None
    wait = 0
    for i, ch in enumerate(line):
        if wait > 0: # still within parentheses, so ignore because the recursion took care of it
            wait -= 1
            continue
        if ch == '(': # recurse the rest
            ch, wait = evaluate(line[i+1:])
        if ch == ')': 
            return firstop, i+1
        if isinstance(ch, int):
            if not firstop:
                firstop = ch
            else:
                firstop = eval(f'{firstop}{operator}{ch}')
        else:
            operator = ch
    return firstop
ans = 0
for line in lines:
    line = line.replace("(","( ").replace(")"," )")
    line = aoc.to_int(line.split())
    ans+= evaluate(line)
ans


21993583522852

In [54]:
# part 2
def findclosing(line):
    count = 0
    for index, i in enumerate(line):
        if i  == "(": count+=1
        if i == ')': count -=1
        if count  == 0: return index

def evaluate(line):
    ans = 0
    while '(' in line: # get rid of all the parenthesis blocks
        first = line.index('(')
        last = findclosing(line[first:])+first    
        line[first:last+1] = [evaluate(line[first+1:last])]
    while '+' in line: # reduce the '+' op_indexations
        op_index = line.index('+')
        line[op_index-1:op_index+2] = [line[op_index-1]+line[op_index+1]]

    while '*' in line: # finally, reduce the '*'
        op_index = line.index('*')
        line[op_index-1:op_index+2] = [line[op_index-1]*line[op_index+1]]
    return line[0]

ans = 0
for line in lines:
    line = line.replace("(","( ").replace(")"," )")
    line = list(aoc.to_int(line.split()))
    ans += evaluate(line)
ans

122438593522757

In [39]:
# alternative solution from reddit, amazing idea with operator overloading
import re
class a(int):
    def __mul__(self, b):
        return a(int(self) + b)
    def __add__(self, b):
        return a(int(self) + b)
    def __sub__(self, b):
        return a(int(self) * b)

def ev(expr, pt2=False):
    expr = re.sub(r"(\d+)", r"a(\1)", expr)
    expr = expr.replace("*", "-")
    if pt2:
        expr = expr.replace("+", "*")
    return eval(expr, {}, {"a": a})

print("Part 1:", sum(ev(l) for l in lines))
print("Part 2:", sum(ev(l, pt2=True) for l in lines))

Part 1: 21993583522852
Part 2: 122438593522757


In [14]:
# another one from sophiebits, have to study the regex a bit

def solve(line):
    def doInner(inner):
        # part 1:
        # while '+' in inner or '*' in inner:
        #     inner = re.sub('^(\d+)\s*\+\s*(\d+)', lambda m: str(int(m.group(1)) + int(m.group(2))), inner)
        #     inner = re.sub('^(\d+)\s*\*\s*(\d+)', lambda m: str(int(m.group(1)) * int(m.group(2))), inner)
        while '+' in inner:
            inner = re.sub('(\d+)\s*\+\s*(\d+)', lambda m: str(int(m.group(1)) + int(m.group(2))), inner)
        while '*' in inner:
            inner = re.sub('(\d+)\s*\*\s*(\d+)', lambda m: str(int(m.group(1)) * int(m.group(2))), inner)
        return inner
    while '(' in line:
        def doExpr(match):
            inner = match.group(1)
            return doInner(inner)
        line = re.sub(r'\(([^()]+)\)', doExpr, line)
    return doInner(line)

total = 0
for line in lines:
    total += int(solve(line))

print(total)

122438593522757
